In [2]:
import numpy as np

import noise3d

# We'll be creating 3D sequences with T=100 frames, V=100 lines, and H=100 columns.
T = 100
V = 100
H = 100
# but you can define any shape

# Create some seq to apply operators on
seq = noise3d.genseq.genseq_3dnoise_seq(T, V, H, [1 for i in range(7)], [0 for i in range(7)])

# Basic noise variances

In [5]:
# Getting the individual noise variances 
print(noise3d.noise.var_nt(seq))
print(noise3d.noise.var_nv(seq))
print(noise3d.noise.var_nh(seq))
print(noise3d.noise.var_ntv(seq))
print(noise3d.noise.var_nth(seq))
print(noise3d.noise.var_nvh(seq))
print(noise3d.noise.var_ntvh(seq))
print()

# Wrapping all in a function, as well as total variance
noise_vars = noise3d.noise.get_all_3d_noise_var_classic(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")



0.8574202462437625
0.9693813306838662
1.1250667653267605
0.9901251786151329
0.9787241634804568
0.9884197482940323
0.9700115600551095

Var(t  ) : 0.86
Var(v  ) : 0.97
Var(h  ) : 1.13
Var(tv ) : 0.99
Var(th ) : 0.98
Var(vh ) : 0.99
Var(tvh) : 0.97
Var(tot) : 6.85


## Faster equivalent version

A faster equivalent version 

In [6]:
noise_vars = noise3d.noise.get_all_3d_noise_var_fast(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")


Var(t  ) : 0.86
Var(v  ) : 0.97
Var(h  ) : 1.13
Var(tv ) : 0.99
Var(th ) : 0.98
Var(vh ) : 0.99
Var(tvh) : 0.94
Var(tot) : 6.85


In [8]:
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_noise_var_classic(seq)

6.55 ms ± 89.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
44.4 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Matrix approach

A matrix unmixin approach is possible to compute the 7 noise variances.
The inputs of this approach are : 
 - a sequence
 - a mixin (or unmixin) matrix
 
The variances can be computed throu the helper, generic function \_get_all_3d_variance_from_matrix.  
The "quality" of the approach depends on the matrix coefficients.  
The classic approach variances can be computed throug the matrix approach using the matrix M_classic:

In [9]:
print(noise3d.noise.M_classic)

[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0]
 [1 0 1 0 1 0 0]
 [0 1 1 0 0 1 0]
 [1 1 1 1 1 1 1]]


In [11]:
sequence_approach = noise3d.noise.get_all_3d_noise_var_classic(seq, names=True)
# using the generic function
matrix_approach_generic = noise3d.noise._get_all_3d_variance_from_matrix(seq, noise3d.noise.M_classic)

for var1, var2, name in zip(sequence_approach[:-1], matrix_approach_generic, sequence_approach[-1]):
    print(f"Variance {name:3} tol : {var1/var2-1:.5f}")

Variance t   tol : 0.00000
Variance v   tol : -0.00000
Variance h   tol : 0.00000
Variance tv  tol : 0.01861
Variance th  tol : 0.02047
Variance vh  tol : 0.02143
Variance tvh tol : -0.02868
Variance tot tol : 0.00000


A wrapped version is avalable.

In [12]:
matrix_approach = noise3d.noise.get_all_3d_classic_var_matrix(seq)
matrix_approach == matrix_approach_generic

True

Then, another matrix, called "corrected" matrix, which is provides unbiased variances estimations. Its values depends on the sequence size.


In [13]:
print(noise3d.noise.compute_M_corrected(T, V, H))

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e-02 1.0000000e-02
  0.0000000e+00 1.0000000e-04]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e-02 0.0000000e+00
  1.0000000e-02 1.0000000e-04]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e-02
  1.0000000e-02 1.0000000e-04]
 [9.9009901e-01 9.9009901e-01 0.0000000e+00 1.0000000e+00 9.9009901e-03
  9.9009901e-03 1.0000000e-02]
 [9.9009901e-01 0.0000000e+00 9.9009901e-01 9.9009901e-03 1.0000000e+00
  9.9009901e-03 1.0000000e-02]
 [0.0000000e+00 9.9009901e-01 9.9009901e-01 9.9009901e-03 9.9009901e-03
  1.0000000e+00 1.0000000e-02]
 [9.9000099e-01 9.9000099e-01 9.9000099e-01 9.9990100e-01 9.9990100e-01
  9.9990100e-01 1.0000000e+00]]


We can then compute variances more "accurately" with this matrix, using a wrapped function :

In [14]:
corrected = noise3d.noise.get_all_3d_corrected_var_matrix(seq)
print(corrected)

(0.837433975757146, 0.9492961456451118, 1.105097893674733, 1.0001306798583365, 0.988499341185586, 0.9983907963994354, 0.9997027617709058, 6.878551594291255)


# Comparison of methods

In [16]:
# in terms of time
%timeit noise3d.noise.get_all_3d_noise_var_classic(seq)
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_classic_var_matrix(seq)
%timeit noise3d.noise.get_all_3d_corrected_var_matrix(seq)

46.2 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.64 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.94 ms ± 466 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.72 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
# values
classic = noise3d.noise.get_all_3d_noise_var_classic(seq, names=True)
names = classic[-1]
classic = classic[:-1]
classic_fast = noise3d.noise.get_all_3d_noise_var_fast(seq)
classic_matrix = noise3d.noise.get_all_3d_classic_var_matrix(seq)
corrected_matrix = noise3d.noise.get_all_3d_corrected_var_matrix(seq)

for cla, claf, clam, corm, name in zip(classic, classic_fast, classic_matrix, corrected_matrix, names):
    print(f"Variance {name}")
    print(cla)
    print(claf)
    print(clam)
    print(corm)
    print()

Variance t
0.8574202462437625
0.8574202462437622
0.8574202462437622
0.837433975757146

Variance v
0.9693813306838662
0.9693813306838662
0.9693813306838664
0.9492961456451118

Variance h
1.1250667653267605
1.1250667653267605
1.1250667653267605
1.105097893674733

Variance tv
0.9901251786151329
0.990125178615133
0.9720380342891172
1.0001306798583365

Variance th
0.9787241634804568
0.9787241634804568
0.9590955792074816
0.988499341185586

Variance vh
0.9884197482940323
0.9884197482940323
0.967682638432541
0.9983907963994354

Variance tvh
0.9700115600551095
0.9402030290524275
0.9986558675129094
0.9997027617709058

Variance tot
6.849340461696438
6.849340461696438
6.849340461696437
6.878551594291255

